In [10]:
import requests
from bs4 import BeautifulSoup
import os
import urllib.request
import geopandas as gpd
import pandas as pd
import re
import shutil
import zipfile

In [11]:
response = requests.get("https://data.gcis.nat.gov.tw/od/datacategory", verify=False)

C:\Users\GIS519\Anaconda3\envs\python38\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gcis.nat.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [12]:
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()) 

<!DOCTYPE html >
<html lang="zh-tw">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- jquery ui includes  -->
  <script src="/resources/jqueryui/js/jquery-3.5.1.min.js;jsessionid=FF60858619F4672604224D0C6DD89EBC" type="text/javascript">
  </script>
  <script src="/resources/jqueryui/js/jquery-ui-1.12.1.min.js;jsessionid=FF60858619F4672604224D0C6DD89EBC" type="text/javascript">
  </script>
  <link href="/resources/jqueryui/css/ui-lightness/jquery-ui-1.12.1.min.css;jsessionid=FF60858619F4672604224D0C6DD89EBC" rel="stylesheet" type="text/css"/>
  <script src="/resources/js/ajax.js;jsessionid=FF60858619F4672604224D0C6DD89EBC" type="text/javascript">
  </script>
  <!-- layout css includes -->
  <link href="/resources/layout/opd/css/component.css;jsessionid=FF60858619F4672604224D0C6DD89EBC" rel="stylesheet" type="text/css"/>
  <link href="/resources/layout/opd/css/rwd-table.min.css;jsessionid=FF60858619F4672604224D0C6DD89EBC" rel="stylesheet" type="text/c

In [13]:
soup.find(id="cate8")

<ul id="cate8">
<li><a href="/od/detail;jsessionid=FF60858619F4672604224D0C6DD89EBC?oid=EFBEB352-3285-42C1-9D2A-CD9AC2D92CFF">商業登記(依營業項目別)－食用油脂批發業</a>
<p class="order">更新日期 │ <span class="date">2021/02/01</span></p>
<p class="order">使用次數 │ <span class="tCount">40</span></p>
</li>
<li><a href="/od/detail;jsessionid=FF60858619F4672604224D0C6DD89EBC?oid=E8D255FA-6CCA-4CBF-9DB1-6A651691FE78">商業登記(依營業項目別)－其他農、畜、水產品批發業</a>
<p class="order">更新日期 │ <span class="date">2021/02/01</span></p>
<p class="order">使用次數 │ <span class="tCount">25</span></p>
</li>
<li><a href="/od/detail;jsessionid=FF60858619F4672604224D0C6DD89EBC?oid=D81B98CD-0A05-4456-B2B5-4D0F019C6BF0">商業登記(依營業項目別)－農產品整理業</a>
<p class="order">更新日期 │ <span class="date">2021/02/01</span></p>
<p class="order">使用次數 │ <span class="tCount">25</span></p>
</li>
<li><a href="/od/detail;jsessionid=FF60858619F4672604224D0C6DD89EBC?oid=B7E47180-09C2-46CC-997F-6248D97135BC">商業登記(依營業項目別)－製酒業</a>
<p class="order">更新日期 │ <span class="date">2021/02/01<

In [14]:
#抓特定區域
a=soup.find(id="cate8")
#抓連結
i=[]
course_links = a.find_all('a', {'href': re.compile('/od/detail;*')})
for link in course_links:
    i.append(link['href'])

In [16]:
len(i)

113

In [17]:
#完整連結網址
weball=[]
for com in i:
    j='https://data.gcis.nat.gov.tw'+ com
    #print(weball)
    weball.append(j)

In [18]:
#onclick=[]
all_data= pd.DataFrame(columns = ["統一編號","商業名稱","商業地址","登記狀態"])#建立空的datafram
for g in weball:
   
    r = requests.get(g)
    soup2 = BeautifulSoup(r.text, "html.parser")
    #print(soup2.prettify())  #輸出排版後的HTML內容
    
    #抓下載zip檔的連結
    a=soup2.find_all("a", {"class": "csv"})[0]
    onclick=a.get('onclick')
    onclick=onclick[23:72]
    zip_links='https://data.gcis.nat.gov.tw'+ onclick
    print(zip_links)
    #把zip檔放入本機
    oldname=list(urllib.request.urlretrieve(zip_links))
    oldname=oldname[0][35:46]
    #設置舊文件名（就是路徑+文件名）
    oldname='C:\\Users\\GIS519\\AppData\\Local\\Temp\\'+oldname
    #設置新文件名 
    newname=oldname+'.zip' 
    #用os模塊中的rename方法對文件改名 
    os.rename(oldname,newname)
    #讀取zip
    myzip=zipfile.ZipFile(os.path.join(os.getcwd(), newname))
    name=myzip.namelist()
    
    #try:
        #download_zip(myzip, path)
    #except False:
        #print("這個檔案錯誤")
        #pass
    try:
        for i in name:
            f=myzip.open(i)
            df=pd.read_csv(f)
            print(df)
    #f.close()
    #myzip.close()
            all_data=all_data.append(df,ignore_index=True)
    except (IOError, zipfile.BadZipfile) as e:
        print("這個檔案錯誤")
        pass
    #df=pd.read_csv(f)
    #print(df)
    #f.close()
    #myzip.close()
    #all_data=all_data.append(df,ignore_index=True)
    
    #break

https://data.gcis.nat.gov.tw/od/file?oid=39084A39-7A6E-455F-91A3-C0DB6EBC85B7
         統一編號     商業名稱                             商業地址   登記狀態
0     8774246     超傑酒莊                 彰化縣二林鎮興華里東川路１１０號  歇業／撤銷
1     8832908  荔園釀酒工作室  南投縣南投市漳和里彰南路２段４７５巷７弄１２號﹝現場不得堆棧﹞   核准設立
2     9143401   美舒容企業社              臺南市佳里區海澄里萊芊寮1之87號1樓  歇業／撤銷
3     9156658     益長酒莊               彰化縣二林鎮西斗里再發路３５之１１號   核准設立
4     9200465  大廟口休閒酒莊             彰化縣埤頭鄉和豊村斗苑東路２４７巷７８號  歇業／撤銷
..        ...      ...                              ...    ...
303  99903769   台灣陸華酒莊           彰化縣芳苑鄉新生村新復路一段三三一巷一二０號  歇業／撤銷
304  99921197     竹峰酒莊                 雲林縣土庫鎮溪邊里溪埔三五之三號   核准設立
305  99921605     泰元酒莊       彰化縣田中鎮沙崙里興酪路１段４２５巷２１１弄１６０號  歇業／撤銷
306  99922082     吉園酒莊                 臺中市外埔區六分里水頭一路３５號     停業
307  99937819     上禾酒莊                雲林縣古坑鄉荷苞村荷苞厝五六之一號  歇業／撤銷

[308 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=3A583B69-6CC4-4AC7-9487-1E06A46C898F
         統一編號            商業名稱                 商業地址   登記狀態
0     

          統一編號        商業名稱                      商業地址   登記狀態
0        51980  狗爸爸寵物用品專賣店        屏東縣東港鎮興東里中正路一段二一一號  歇業／撤銷
1       182509       翔園企業社            高雄市美濃區興南街２５號１樓  歇業／撤銷
2       238047     鋐運環保企業社         高雄市仁武區名湖街５１巷１２號１樓  歇業／撤銷
3       656040    奈斯寵物精品名店          高雄市苓雅區河北路２８巷１之２號  歇業／撤銷
4      1192049        文非商行        金門縣金湖鎮正義里夏興85號3樓之9   核准設立
...        ...         ...                       ...    ...
2775  99907337       龍華企業社      苗栗縣苗栗市玉清里17鄰復興路5段18號  歇業／撤銷
2776  99907651      悠活水族工坊    臺中市西屯區何安里西屯路二段１０３之２號１樓  歇業／撤銷
2777  99937058    隆陽寵物百貨商行  臺中市南屯區田心里五權西路二段３８９巷２１號１樓  歇業／撤銷
2778  99961850     賜海水族企業社          臺北市松山區民權東路5段3號1樓   核准設立
2779  99968880     三享設計工作室      臺北市大安區敦化南路2段76號17樓之2  歇業／撤銷

[2780 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=12DF9939-1B72-4C88-AD7E-2D182DEC1462
         統一編號      商業名稱                            商業地址   登記狀態
0      627003      騰雲茶行               高雄市楠梓區建楠路８４巷７１號１樓   核准設立
1      912301   永福土木包工業            基隆市安樂區永康里崇德路一○巷六

https://data.gcis.nat.gov.tw/od/file?oid=587A0294-3CC6-49EA-BB1C-44D6DA848BD4
           統一編號     商業名稱                  商業地址   登記狀態
0         35783    六釆軒商行       臺中市中區中華路二段１８號一樓   核准設立
1        144647  快樂壹百遊樂園         高雄市三民區澄和路157號  歇業／撤銷
2        145483    春誠企業行      高雄市前鎮區和平二路１４０號１樓   核准設立
3        146188    兆馥企業社       高雄市楠梓區加宏路２３５號１樓     停業
4        147460    開得發商行  高雄市苓雅區城東里仁義街２３號十三樓之１  歇業／撤銷
...         ...      ...                   ...    ...
25564  99987015     承鑫商號     臺北市大安區和平東路3段10號6樓  歇業／撤銷
25565  99991165    師富企業社       臺北市松山區三民路152號1樓  歇業／撤銷
25566  99991186     維慶商行  臺北市中山區新生北路3段62巷16號1樓  歇業／撤銷
25567  99993797   新桂冠食品行      臺北市南港區忠孝東路6段432號  歇業／撤銷
25568  99994070    偉翔企業社  臺北市大同區重慶北路3段25巷30號1樓   核准設立

[25569 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=851B0668-ACAA-4B8C-AB25-951D1B20DADA
           統一編號    商業名稱                  商業地址   登記狀態
0         11702   大成報關行     基隆市仁愛區文昌里忠二路五二號四樓   核准設立
1         28976     俊宏行   臺中市西區民龍里大仁街６０巷１４號一樓   核准設立
2        10

         統一編號       商業名稱                       商業地址   登記狀態
0     1744985  環保尖兵國際實業社       桃園市楊梅區裕新里裕成南路276巷45號  歇業／撤銷
1     2393488    家簡塵除企業社        新北市深坑區北深路2段202巷7號4樓   核准設立
2     6427185   龍吉中西日餐具行       臺南市安平區天妃里民權路4段763號1樓   核准設立
3     8722206      寶華企業社        新北市新莊區昌隆里昌平街59巷3號2樓   核准設立
4     8968284      芊富企業社            臺南市官田區社子里社子9號1樓   核准設立
..        ...        ...                        ...    ...
342  99298572        驛馬行  新北市三重區慈愛街４４號３樓（現場僅供辦公室使用）  歇業／撤銷
343  99497328      博楷企業社          花蓮縣光復鄉大華村仁愛路３８號１樓  歇業／撤銷
344  99735320     齊立潔企業行      屏東縣新園鄉鹽埔村鹽洲路377巷5之10號     停業
345  99735320     齊立潔企業行         高雄市小港區桂林里中安路４３８號４樓   遷他縣市
346  99735813    立代清潔工程行          高雄市三民區義昌路５８號５樓之１２  歇業／撤銷

[347 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=6D2611AD-A2C3-40ED-B7CC-B10CBF07E31A
           統一編號       商業名稱                     商業地址   登記狀態
0         63905      天儀鐘錶行        基隆市仁愛區新店里孝三路六六號一樓   核准設立
1         63905     豪聲影視商行        臺中市北區健行里忠明路４５６號一樓  歇業／撤銷
2        1454

C:\Users\GIS519\Anaconda3\envs\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


...

            統一編號     商業名稱                  商業地址   登記狀態
0            696    義達什貨店          新北市新莊區自治街７７號   核准設立
1           1107    一新切麵店          基隆市仁愛區忠四路35號  歇業／撤銷
2           1280  福興食品原料廠   臺中市西區後龍里中山路４７１之１號１樓   核准設立
3           1514    五洲豆漿店    基隆市中正區信義里義二路一六一號之六   核准設立
4           2534   三文珍食品店       基隆市仁愛區花崗里愛九路一六號  歇業／撤銷
...          ...      ...                   ...    ...
213270  BP901403      頂昌行   臺東縣臺東市大同里十八鄰廣東路一七三號  歇業／撤銷
213271  BP902437      夏興號      臺東縣臺東市岩灣里興安路三二三號  歇業／撤銷
213272  BP903320     春慕商行       臺東縣臺東市中興路三段四三六號  歇業／撤銷
213273  BU005093    郁品居茶藝  臺中市西區平和里柳川東路二段１９６號一樓  歇業／撤銷
213274  H8999642     龍興商店      南投縣中寮鄉龍岩村龍南路二０二號   核准設立

[213275 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=816CCEAC-D50C-4A09-B7E9-A0B278BC370F
           統一編號       商業名稱                     商業地址   登記狀態
0      00004300      大方傢俱行        基隆市信義區義六路28號20樓之1   核准設立
1      00030004      大榮鐵材行         基隆市信義區東信路一八Ｏ巷六四號   核准設立
2      00037359      中興建材行             基隆市中山區中華路二三號  歇業／撤銷

https://data.gcis.nat.gov.tw/od/file?oid=D92DD6D7-4F7D-4391-9750-1F033406CA68
          統一編號      商業名稱                   商業地址   登記狀態
0     00711769       張文誠     基隆市暖暖區源遠路二六Ｏ巷三四弄八號   核准設立
1     00829937       劉家祿       基隆市信義區信二路一七四巷九一號   核准設立
2     01475617  福運小客車租賃行  桃園市楊梅區裕新里裕成南路374號4樓之1   核准設立
3     01482899  凱禾小客車租賃行      桃園市大園區北港里自立四街5之1號   核准設立
4     01482899  凱禾小客車租賃行    新北市泰山區泰林路2段420號(1樓)   遷他縣市
...        ...       ...                    ...    ...
3146  A9902277      廖劉快妹       新竹縣竹東鎮商華里康寧街五十一號  歇業／撤銷
3147  A9902338    四季香飲食店     新竹縣竹東鎮東華里東林路二十七巷二號  歇業／撤銷
3148  A9902585   林華土木包工業       新竹縣新埔鎮新民里中正路三二五號  歇業／撤銷
3149  A9903056      日文鞋店       新竹縣竹北市竹仁里中正路八十七號   核准設立
3150  A9903250     義豊焦煤行       新竹縣竹北市竹義里和平路二十六號  歇業／撤銷

[3151 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=C855E073-2659-4A3C-9086-9B93600A894C
          統一編號         商業名稱                     商業地址   登記狀態
0       143318        花都殯葬社            基隆市仁愛區愛四路八號一樓   核准設立
1       162200        憶安禮儀社      高雄市前金區

https://data.gcis.nat.gov.tw/od/file?oid=5F4BB71A-B234-43C8-B83E-A99F77F1AEB1
       統一編號      商業名稱                      商業地址   登記狀態
0  41006773    尼亞諾企業社       花蓮縣光復鄉北富村光豐路57巷1號1樓     廢止
1  42274396  喬凱立文化創意坊      臺北市大安區忠孝東路4段59號10樓之2   核准設立
2  85321046    天然國際商行        宜蘭縣三星鄉拱照村三星路三段２６８號  歇業／撤銷
3  87108669  珍善媒體行銷商行   臺中市西屯區潮洋里市政北二路２３６號３０樓之５   核准設立
4  87382883     起頭工作室  宜蘭縣宜蘭市成功里東港路二段292巷6弄1號一樓   核准設立
https://data.gcis.nat.gov.tw/od/file?oid=41C68DA1-0090-460F-BA9E-18ADD785D5C2
         統一編號             商業名稱                  商業地址   登記狀態
0      431401   台灣鐵路管理局基隆貨運服務所          基隆市仁愛區港西街五五號   核准設立
1      673343  金利交通企業有限公司基隆連絡處        基隆市中正區中船路五巷五六號   核准設立
2      682125            順祥搬運行        基隆市仁愛區港西街一九號二樓  歇業／撤銷
3      914997     林潘個人經營小貨車貨運行       基隆市仁愛區獅球路二五巷三五號   核准設立
4      916670    顏寶護個人經營小貨車貨運行        基隆市仁愛區華三街二九號二樓  歇業／撤銷
..        ...              ...                   ...    ...
716  98167453            萬鈞起重行     苗栗縣苑裡鎮山腳里十六鄰錦山一三號  歇業／撤銷
717  98730332            時運貨運行  臺中

https://data.gcis.nat.gov.tw/od/file?oid=068D420C-D346-401C-B995-561E984729D7
          統一編號   商業名稱                     商業地址   登記狀態
0     00003089   順天葯局       彰化縣鹿港鎮廖厝里鹿和路二段２８７號   核准設立
1     00037409  豪星廚具行         臺中市西區後龍里五權路７９號一樓  歇業／撤銷
2     00037409   仁生藥局              基隆市仁愛區仁三路2號   核准設立
3     00051400  文通西藥房          基隆市仁愛區玉田里仁三路四Ｏ號  歇業／撤銷
4     00055095   師大藥局      屏東縣屏東市橋北里廣東路四五之二號一樓   核准設立
...        ...    ...                      ...    ...
8230  99926138   康好藥局  臺中市大甲區孔門里三民路一三二之二號（限一樓）   核准設立
8231  99935116   怡安藥局       苗栗縣頭份市東庄里自強路一０三號一樓   核准設立
8232  BD024588   海山藥局          新北市新莊區景德路425號1樓   核准設立
8233  BG000179  大盛西藥房     苗栗縣通霄鎮平元里一九鄰南和路一段三一號     廢止
8234  BG024296   台榮葯局        苗栗縣苗栗市青苗里六鄰至公路二一號   核准設立

[8235 rows x 4 columns]
https://data.gcis.nat.gov.tw/od/file?oid=0041580E-F9C8-404E-AD44-03A43D0898C6
          統一編號      商業名稱                  商業地址   登記狀態
0       169105     正一實業社        高雄市前金區河南二路１２１號   核准設立
1       238530  大東不動產企業社      高雄市鳳山區大東一路110號１樓   核准設立
2       2

C:\Users\GIS519\Anaconda3\envs\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
all_data

,統一編號,商業名稱,商業地址,登記狀態
0,8774246,超傑酒莊,彰化縣二林鎮興華里東川路１１０號,歇業／撤銷
1,8832908,荔園釀酒工作室,南投縣南投市漳和里彰南路２段４７５巷７弄１２號﹝現場不得堆棧﹞,核准設立
2,9143401,美舒容企業社,臺南市佳里區海澄里萊芊寮1之87號1樓,歇業／撤銷
3,9156658,益長酒莊,彰化縣二林鎮西斗里再發路３５之１１號,核准設立
4,9200465,大廟口休閒酒莊,彰化縣埤頭鄉和豊村斗苑東路２４７巷７８號,歇業／撤銷
...,...,...,...,...
2627508,99994655,洋洋小吃店,臺北市大同區涼州街72巷12號,歇業／撤銷
2627509,99999841,嘉富飲食店,新竹市北區南寮里４號,核准設立
2627510,BI000983,貴族便當行,彰化縣員林市惠來里惠明街１０６巷７６弄７號,停業
2627511,BL013369,清淙幻麵店,嘉義縣朴子市安福中正三一,核准設立


In [31]:
c=all_data[all_data["登記狀態"]=="核准設立"]
c

,統一編號,商業名稱,商業地址,登記狀態
1,8832908,荔園釀酒工作室,南投縣南投市漳和里彰南路２段４７５巷７弄１２號﹝現場不得堆棧﹞,核准設立
3,9156658,益長酒莊,彰化縣二林鎮西斗里再發路３５之１１號,核准設立
8,9282281,華山泉酒莊,雲林縣斗六市湖山里湖山路一三五之三五號,核准設立
9,9288476,海馬酒莊,彰化縣伸港鄉海尾村海尾路６０之１３號１樓,核准設立
10,9315115,月眉酒庒,臺中市外埔區六分里月眉西路１５０巷３５號,核准設立
...,...,...,...,...
2627483,99993136,魔椅商行,臺北市中正區紹興北街31巷1號,核准設立
2627497,99993895,鮮定味小吃店,臺北市中山區長安東路1段67號1樓,核准設立
2627509,99999841,嘉富飲食店,新竹市北區南寮里４號,核准設立
2627511,BL013369,清淙幻麵店,嘉義縣朴子市安福中正三一,核准設立


In [32]:
v=all_data[all_data["登記狀態"]=="遷他縣市"]
v

,統一編號,商業名稱,商業地址,登記狀態
18,10616027,吉祥食品工廠,新北市土城區慶祥街１１０號（１樓）,遷他縣市
56,14642130,柑園食品廠,新北市樹林區東興街１６巷２８號,遷他縣市
93,15541320,民生食品開發工廠,桃園市桃園區建國里吉林路７號,遷他縣市
317,8708605,澤鬘企業社,臺北市松山區八德路3段27號2樓,遷他縣市
319,10515132,信德土木包工業,桃園市龍潭區上林里溝西６巷１２號１樓,遷他縣市
...,...,...,...,...
2627189,99965194,蒲善屋日式餐坊,臺北市中山區錦州街459號1樓,遷他縣市
2627212,99966193,益宸商行,臺北市松山區八德路2段386號6樓之1,遷他縣市
2627217,99966460,東聯企業社,臺北市中山區天津街31號1樓,遷他縣市
2627266,99972532,極品宴餐廳,臺北市大安區光復南路410號1樓,遷他縣市


In [33]:
e=c.append(v)
e

,統一編號,商業名稱,商業地址,登記狀態
1,8832908,荔園釀酒工作室,南投縣南投市漳和里彰南路２段４７５巷７弄１２號﹝現場不得堆棧﹞,核准設立
3,9156658,益長酒莊,彰化縣二林鎮西斗里再發路３５之１１號,核准設立
8,9282281,華山泉酒莊,雲林縣斗六市湖山里湖山路一三五之三五號,核准設立
9,9288476,海馬酒莊,彰化縣伸港鄉海尾村海尾路６０之１３號１樓,核准設立
10,9315115,月眉酒庒,臺中市外埔區六分里月眉西路１５０巷３５號,核准設立
...,...,...,...,...
2627189,99965194,蒲善屋日式餐坊,臺北市中山區錦州街459號1樓,遷他縣市
2627212,99966193,益宸商行,臺北市松山區八德路2段386號6樓之1,遷他縣市
2627217,99966460,東聯企業社,臺北市中山區天津街31號1樓,遷他縣市
2627266,99972532,極品宴餐廳,臺北市大安區光復南路410號1樓,遷他縣市


In [37]:
e.to_csv('商業登記名冊(只留核准設立與遷市).csv', index=False,encoding="utf_8_sig" )

In [38]:
e

,統一編號,商業名稱,商業地址,登記狀態
1,8832908,荔園釀酒工作室,南投縣南投市漳和里彰南路２段４７５巷７弄１２號﹝現場不得堆棧﹞,核准設立
3,9156658,益長酒莊,彰化縣二林鎮西斗里再發路３５之１１號,核准設立
8,9282281,華山泉酒莊,雲林縣斗六市湖山里湖山路一三五之三五號,核准設立
9,9288476,海馬酒莊,彰化縣伸港鄉海尾村海尾路６０之１３號１樓,核准設立
10,9315115,月眉酒庒,臺中市外埔區六分里月眉西路１５０巷３５號,核准設立
...,...,...,...,...
2627189,99965194,蒲善屋日式餐坊,臺北市中山區錦州街459號1樓,遷他縣市
2627212,99966193,益宸商行,臺北市松山區八德路2段386號6樓之1,遷他縣市
2627217,99966460,東聯企業社,臺北市中山區天津街31號1樓,遷他縣市
2627266,99972532,極品宴餐廳,臺北市大安區光復南路410號1樓,遷他縣市


In [41]:
ff=e.drop_duplicates("統一編號",keep='last', inplace=False)
ff

,統一編號,商業名稱,商業地址,登記狀態
3,9156658,益長酒莊,彰化縣二林鎮西斗里再發路３５之１１號,核准設立
9,9288476,海馬酒莊,彰化縣伸港鄉海尾村海尾路６０之１３號１樓,核准設立
10,9315115,月眉酒庒,臺中市外埔區六分里月眉西路１５０巷３５號,核准設立
11,9330832,老長壽養生酒坊,彰化縣溪州鄉瓦厝村東安路１１號,核准設立
12,9518619,最遊釀酒坊,屏東縣長治鄉榮華村和平路３１－１號一樓,核准設立
...,...,...,...,...
2627189,99965194,蒲善屋日式餐坊,臺北市中山區錦州街459號1樓,遷他縣市
2627212,99966193,益宸商行,臺北市松山區八德路2段386號6樓之1,遷他縣市
2627217,99966460,東聯企業社,臺北市中山區天津街31號1樓,遷他縣市
2627266,99972532,極品宴餐廳,臺北市大安區光復南路410號1樓,遷他縣市


In [42]:
ff.to_csv('商業登記名冊(只留核准設立OR遷市).csv', index=False,encoding="utf_8_sig" )

In [95]:
all_data

,統一編號,商業名稱,商業地址,登記狀態
0,8774246,超傑酒莊,彰化縣二林鎮興華里東川路１１０號,歇業／撤銷
1,8832908,荔園釀酒工作室,南投縣南投市漳和里彰南路２段４７５巷７弄１２號﹝現場不得堆棧﹞,核准設立
2,9143401,美舒容企業社,臺南市佳里區海澄里萊芊寮1之87號1樓,歇業／撤銷
3,9156658,益長酒莊,彰化縣二林鎮西斗里再發路３５之１１號,核准設立
4,9200465,大廟口休閒酒莊,彰化縣埤頭鄉和豊村斗苑東路２４７巷７８號,歇業／撤銷
...,...,...,...,...
2616071,99994655,洋洋小吃店,臺北市大同區涼州街72巷12號,歇業／撤銷
2616072,99999841,嘉富飲食店,新竹市北區南寮里４號,核准設立
2616073,BI000983,貴族便當行,彰化縣員林市惠來里惠明街１０６巷７６弄７號,停業
2616074,BL013369,清淙幻麵店,嘉義縣朴子市安福中正三一,核准設立


In [96]:
all_data.to_csv('商業登記名冊.csv', index=False,encoding="utf_8_sig" )

空檔案(商業登記(依營業項目別)－化粧品零售業):
https://data.gcis.nat.gov.tw/od/file?oid=266BC62A-700A-4794-8A98-0700462E63E6
空欄位(商業登記(依營業項目別)－加氣站業):
https://data.gcis.nat.gov.tw/od/file?oid=A69A91F3-3357-4CE1-9184-108BD8CCEBC9

In [89]:
#有壓縮檔解開後沒有任何csv資料，所以設定錯誤警告來跳過此檔
try:
    for i in name:
        f=myzip.open(i)
        df=pd.read_csv(f)
        print(df)
    #f.close()
    #myzip.close()
        all_data=all_data.append(df,ignore_index=True)
except (IOError, zipfile.BadZipfile) as e:
    print("這個檔案錯誤")
    pass

這個檔案錯誤


def download_zip(z, path):
    if not(z.testzip()):
        z.extractall(path)
        return True
    else:
        return False

path='C:\\Users\\GIS519\\AppData\\Local\\Temp\\'

download_zip(myzip, path)

def download_zip(z, path):
    names_files = z.namelist()
    count = 0
    for my_file in names_files:
        if my_file:
            if z.testzip():
                if not(my_file in z.testzip()):
                    try:
                        z.extract(my_file, path=path)
                    except zipfile.BadZipfile:
                        count = count +1
            else:
                z.extract(my_file, path=path)
        else:
            count = count + 1

    if count != 0:
        my_path = os.getcwd()
        parent = os.path.dirname(my_path)
        os.chdir(parent)
        os.rename(my_path, my_path + ' - ' + str(count) + ' doc du zip non extrait')
        os.chdir(my_path + ' - ' + str(count) + ' doc du zip non extrait')
    return count